In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mesa.space import MultiGrid
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import pandas as pd

In [4]:
dj = pd.read_csv('DJIA (daily).csv',index_col='DATE', parse_dates=True)
vix = pd.read_csv('vix (daily).csv',index_col='DATE', parse_dates=True)
vix=vix[1:] #Removing first row 
dj=dj[:-1] #Removing last row
dj['DJIA'] = dj['DJIA'].apply(pd.to_numeric,errors='coerce')
vix['VXDCLS']=vix['VXDCLS'].apply(pd.to_numeric,errors='coerce')
dj=dj.dropna()
vix=vix.dropna()
vix['change']=vix.pct_change(1)
dj['change']= dj.pct_change(1)
biggest=dj['change'].max()
lowest=dj['change'].min()

In [5]:
class FinancialAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model,agent_type):
        super().__init__(unique_id, model)
        self.type = agent_type
        self.number_of_shares = 1
        self.propensity_to_sentiment_contagion = random.uniform(0, 1)
        self.news_reaction=0
        self.my_sentiment=self.news_reaction
        self.a=0.5
        self.b=0.5
        self.wtp=0
       
    def step(self):
        #reaction to news
        x=self.model.run
        if self.type == 1:
            if(dj.iloc[x,1]>0 and vix.iloc[x,1]<0):
                self.news_reaction=1
            elif(dj.iloc[x,1]<0 and vix.iloc[x,1]<0):
                self.news_reaction= -0.25
            elif(dj.iloc[x,1]>0 and vix.iloc[x,1]>0):
                self.news_reaction= 0.25
            elif(dj.iloc[x,1]<0 and vix.iloc[x,1]>0):
                self.news_reaction= -1

        elif self.type == 0:
            if(dj.iloc[x,1]>0 and vix.iloc[x,1]<0):
                self.news_reaction=-1
            elif(dj.iloc[x,1]<0 and vix.iloc[x,1]<0):
                self.news_reaction= -1
            elif(dj.iloc[x,1]>0 and vix.iloc[x,1]>0):
                self.news_reaction= np.random.choice([-1,1])
            elif(dj.iloc[x,1]<0 and vix.iloc[x,1]>0):
                self.news_reaction= 1
                
        #updating market sentiment
        if (self.model.run != 0):
            if (self.model.returns> 0 and self.news_reaction>0):
                if self.a>self.b:
                    if self.propensity_to_sentiment_contagion + self.model.returns <=1:
                        self.propensity_to_sentiment_contagion += self.model.returns
                else:
                    if self.propensity_to_sentiment_contagion - self.model.returns >=-1:
                        self.propensity_to_sentiment_contagion -= self.model.returns
            elif (self.model.returns> 0 and self.news_reaction<0):
                if self.a>self.b:
                    if self.propensity_to_sentiment_contagion + self.model.returns <=1:
                        self.propensity_to_sentiment_contagion += self.model.returns
                else:
                    if self.propensity_to_sentiment_contagion - self.model.returns >=-1:
                        self.propensity_to_sentiment_contagion -= self.model.returns
            elif (self.model.returns < 0 and self.news_reaction<0):
                if self.a>self.b:
                    if self.propensity_to_sentiment_contagion + self.model.returns >=-1:
                        self.propensity_to_sentiment_contagion += self.model.returns
                else:
                    if self.propensity_to_sentiment_contagion - self.model.returns <=1:
                        self.propensity_to_sentiment_contagion -= self.model.returns
            elif (self.model.returns < 0 and self.news_reaction>0):
                if self.a>self.b:
                    if self.propensity_to_sentiment_contagion + self.model.returns >=-1:
                        self.propensity_to_sentiment_contagion += self.model.returns
                else:
                    if self.propensity_to_sentiment_contagion - self.model.returns <=1:                   
                        self.propensity_to_sentiment_contagion -= self.model.returns
        
        #Decision making after contagion        
        sentiment= [agent.my_sentiment for agent in self.model.grid.get_neighbors(self.pos,moore=True,include_center=False)]
        if len(sentiment)==0:
            c=0
        else:
            self.a= sum([1 for x in sentiment if x >= 0])/len(sentiment)
            self.b= sum([1 for x in sentiment if x < 0])/len(sentiment)  
            c= np.random.choice([1,-1],p=[self.a, self.b])
            x= self.propensity_to_sentiment_contagion*c+ self.news_reaction + np.random.normal(0,1)
                                    
        if(x > 0):
            self.my_sentiment= 1
            self.wtp =x*biggest
            
        else:
            self.my_sentiment = -1
            self.wtp =-x*lowest
            
        
                    

In [6]:
def market_clearing(model):

    number_of_traders= model.num_agents
    if sum([agent.my_sentiment for agent in model.schedule.agents])== 0:
        returns=0
    else:      
        if sum([agent.my_sentiment for agent in model.schedule.agents])> 0:
            returns=0
            while True:
                returns +=0.01
                number_of_shares=[agent.number_of_shares+1 if agent.my_sentiment==1 and returns<= agent.wtp \
                              else agent.number_of_shares-1  if agent.my_sentiment==-1 and returns >= agent.wtp\
                              else agent.number_of_shares \
                              for agent in model.schedule.agents ]
            
                if sum(number_of_shares)<= number_of_traders:
                    break
            
        elif sum([agent.my_sentiment for agent in model.schedule.agents]) < 0:
            returns=0
            while True:
                returns -=0.01
                number_of_shares=[agent.number_of_shares+1 if agent.my_sentiment==1 and returns<= agent.wtp \
                              else agent.number_of_shares-1  if agent.my_sentiment==-1 and returns >= agent.wtp\
                              else agent.number_of_shares \
                              for agent in model.schedule.agents ]
            
                if sum(number_of_shares)>= number_of_traders:
                    break
        for i in range(len(model.schedule.agents)):
            model.schedule.agents[i].number_of_shares=number_of_shares[i]
    
    return returns
   

In [7]:
def price(model):
    try:
        price =model.price*(1+ model.returns)
    except:
         price= dj.iloc[0,0]*(1 + model.returns)
    return price

In [8]:
class FinancialMarket(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height,speculator_pc):
        self.running = True
        self.num_agents = N
        self.speculator_pc= speculator_pc
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.returns=0
        self.run=0
        
        # Create agents
        for i in range(self.num_agents):
            if random.random() < self.speculator_pc:
                agent_type = 0
            else:
                agent_type = 1
            a = FinancialAgent(i, self,agent_type)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            
        self.datacollector = DataCollector(
            model_reporters={"Returns":market_clearing,"price":price}
           )
     
        
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        a=self.datacollector.get_model_vars_dataframe()
        self.returns=a.iloc[-1,0]
        self.price=a.iloc[-1,1]
        self.run +=1       
        if self.run > len(dj.index):
            self.running = False
       
        
       

In [ ]:
model = FinancialMarket(50, 10, 10,0.2)
for i in range(len(dj.index)):
    model.step()

In [ ]:
output = model.datacollector.get_model_vars_dataframe()
output.index= dj.index
output['actual return']=dj['change']
output.to_csv('model output.csv')


In [9]:
# server.py
#Plot agents in their grid and show their movement
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import ChartModule
from mesa.visualization.UserParam import UserSettableParameter



n_slider1 = UserSettableParameter('slider', "Number of Agents", 100, 50, 20000, 100)
n_slider2 = UserSettableParameter('slider', "Width of Grid", 10, 10, 100, 10)
n_slider3 = UserSettableParameter('slider', "Height of Grid", 10, 10, 100, 10)
n_slider4 = UserSettableParameter('slider', "Proportion of Speculators", 0.2, 0, 1, .05)




chart1 = ChartModule([{"Label": "Returns",
                      "Color": "Black"}],
                    data_collector_name='datacollector')
chart2= ChartModule([{"Label": "price",
                      "Color": "Grey"}],data_collector_name='datacollector')

def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    if agent.my_sentiment > 0:
        portrayal["Color"] = "green"
    else:
        portrayal["Color"] = "red"
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(FinancialMarket,
                       [grid,chart1,chart2],
                       "Simulated Financial Market",
                       {"N": n_slider1, "width": n_slider2, "height": n_slider3,"speculator_pc":n_slider4})

In [10]:
# run.py
#from server import server
server.port = 8549 # The default
server.launch()

Interface starting at http://127.0.0.1:8549
Socket opened!
{"type":"get_params"}
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"ge

{"type":"get_step","step":266}
{"type":"get_step","step":267}
{"type":"get_step","step":268}
{"type":"get_step","step":269}
{"type":"get_step","step":270}
{"type":"get_step","step":271}
{"type":"get_step","step":272}
{"type":"get_step","step":273}
{"type":"get_step","step":274}
{"type":"get_step","step":275}
{"type":"get_step","step":276}
{"type":"get_step","step":277}
{"type":"get_step","step":278}
{"type":"get_step","step":279}
{"type":"get_step","step":280}
{"type":"get_step","step":281}
{"type":"get_step","step":282}
{"type":"get_step","step":283}
{"type":"get_step","step":284}
{"type":"get_step","step":285}
{"type":"get_step","step":286}
{"type":"get_step","step":287}
{"type":"get_step","step":288}
{"type":"get_step","step":289}
{"type":"get_step","step":290}
{"type":"get_step","step":291}
{"type":"get_step","step":292}
{"type":"get_step","step":293}
{"type":"get_step","step":294}
{"type":"get_step","step":295}
{"type":"get_step","step":296}
{"type":"get_step","step":297}
{"type":

{"type":"get_step","step":531}
{"type":"get_step","step":532}
{"type":"get_step","step":533}
{"type":"get_step","step":534}
{"type":"get_step","step":535}
{"type":"get_step","step":536}
{"type":"get_step","step":537}
{"type":"get_step","step":538}
{"type":"get_step","step":539}
{"type":"get_step","step":540}
{"type":"get_step","step":541}
{"type":"get_step","step":542}
{"type":"get_step","step":543}
{"type":"get_step","step":544}
{"type":"get_step","step":545}
{"type":"get_step","step":546}
{"type":"get_step","step":547}
{"type":"get_step","step":548}
{"type":"get_step","step":549}
{"type":"get_step","step":550}
{"type":"get_step","step":551}
{"type":"get_step","step":552}
{"type":"get_step","step":553}
{"type":"get_step","step":554}
{"type":"get_step","step":555}
{"type":"get_step","step":556}
{"type":"get_step","step":557}
{"type":"get_step","step":558}
{"type":"get_step","step":559}
{"type":"get_step","step":560}
{"type":"get_step","step":561}
{"type":"get_step","step":562}
{"type":

{"type":"get_step","step":796}
{"type":"get_step","step":797}
{"type":"get_step","step":798}
{"type":"get_step","step":799}
{"type":"get_step","step":800}
{"type":"get_step","step":801}
{"type":"get_step","step":802}
{"type":"get_step","step":803}
{"type":"get_step","step":804}
{"type":"get_step","step":805}
{"type":"get_step","step":806}
{"type":"get_step","step":807}
{"type":"get_step","step":808}
{"type":"get_step","step":809}
{"type":"get_step","step":810}
{"type":"get_step","step":811}
{"type":"get_step","step":812}
{"type":"get_step","step":813}
{"type":"get_step","step":814}
{"type":"get_step","step":815}
{"type":"get_step","step":816}
{"type":"get_step","step":817}
{"type":"get_step","step":818}
{"type":"get_step","step":819}
{"type":"get_step","step":820}
{"type":"get_step","step":821}
{"type":"get_step","step":822}
{"type":"get_step","step":823}
{"type":"get_step","step":824}
{"type":"get_step","step":825}
{"type":"get_step","step":826}
{"type":"get_step","step":827}
{"type":

{"type":"get_step","step":1059}
{"type":"get_step","step":1060}
{"type":"get_step","step":1061}
{"type":"get_step","step":1062}
{"type":"get_step","step":1063}
{"type":"get_step","step":1064}
{"type":"get_step","step":1065}
{"type":"get_step","step":1066}
{"type":"get_step","step":1067}
{"type":"get_step","step":1068}
{"type":"get_step","step":1069}
{"type":"get_step","step":1070}
{"type":"get_step","step":1071}
{"type":"get_step","step":1072}
{"type":"get_step","step":1073}
{"type":"get_step","step":1074}
{"type":"get_step","step":1075}
{"type":"get_step","step":1076}
{"type":"get_step","step":1077}
{"type":"get_step","step":1078}
{"type":"get_step","step":1079}
{"type":"get_step","step":1080}
{"type":"get_step","step":1081}
{"type":"get_step","step":1082}
{"type":"get_step","step":1083}
{"type":"get_step","step":1084}
{"type":"get_step","step":1085}
{"type":"get_step","step":1086}
{"type":"get_step","step":1087}
{"type":"get_step","step":1088}
{"type":"get_step","step":1089}
{"type":

{"type":"get_step","step":1316}
{"type":"get_step","step":1317}
{"type":"get_step","step":1318}
{"type":"get_step","step":1319}
{"type":"get_step","step":1320}
{"type":"get_step","step":1321}
{"type":"get_step","step":1322}
{"type":"get_step","step":1323}
{"type":"get_step","step":1324}
{"type":"get_step","step":1325}
{"type":"get_step","step":1326}
{"type":"get_step","step":1327}
{"type":"get_step","step":1328}
{"type":"get_step","step":1329}
{"type":"get_step","step":1330}
{"type":"get_step","step":1331}
{"type":"get_step","step":1332}
{"type":"get_step","step":1333}
{"type":"get_step","step":1334}
{"type":"get_step","step":1335}
{"type":"get_step","step":1336}
{"type":"get_step","step":1337}
{"type":"get_step","step":1338}
{"type":"get_step","step":1339}
{"type":"get_step","step":1340}
{"type":"get_step","step":1341}
{"type":"get_step","step":1342}
{"type":"get_step","step":1343}
{"type":"get_step","step":1344}
{"type":"get_step","step":1345}
{"type":"get_step","step":1346}
{"type":

{"type":"get_step","step":1573}
{"type":"get_step","step":1574}
{"type":"get_step","step":1575}
{"type":"get_step","step":1576}
{"type":"get_step","step":1577}
{"type":"get_step","step":1578}
{"type":"get_step","step":1579}
{"type":"get_step","step":1580}
{"type":"get_step","step":1581}
{"type":"get_step","step":1582}
{"type":"get_step","step":1583}
{"type":"get_step","step":1584}
{"type":"get_step","step":1585}
{"type":"get_step","step":1586}
{"type":"get_step","step":1587}
{"type":"get_step","step":1588}
{"type":"get_step","step":1589}
{"type":"get_step","step":1590}
{"type":"get_step","step":1591}
{"type":"get_step","step":1592}
{"type":"get_step","step":1593}
{"type":"get_step","step":1594}
{"type":"get_step","step":1595}
{"type":"get_step","step":1596}
{"type":"get_step","step":1597}
{"type":"get_step","step":1598}
{"type":"get_step","step":1599}
{"type":"get_step","step":1600}
{"type":"get_step","step":1601}
{"type":"get_step","step":1602}
{"type":"get_step","step":1603}
{"type":

{"type":"get_step","step":1830}
{"type":"get_step","step":1831}
{"type":"get_step","step":1832}
{"type":"get_step","step":1833}
{"type":"get_step","step":1834}
{"type":"get_step","step":1835}
{"type":"get_step","step":1836}
{"type":"get_step","step":1837}
{"type":"get_step","step":1838}
{"type":"get_step","step":1839}
{"type":"get_step","step":1840}
{"type":"get_step","step":1841}
{"type":"get_step","step":1842}
{"type":"get_step","step":1843}
{"type":"get_step","step":1844}
{"type":"get_step","step":1845}
{"type":"get_step","step":1846}
{"type":"get_step","step":1847}
{"type":"get_step","step":1848}
{"type":"get_step","step":1849}
{"type":"get_step","step":1850}
{"type":"get_step","step":1851}
{"type":"get_step","step":1852}
{"type":"get_step","step":1853}
{"type":"get_step","step":1854}
{"type":"get_step","step":1855}
{"type":"get_step","step":1856}
{"type":"get_step","step":1857}
{"type":"get_step","step":1858}
{"type":"get_step","step":1859}
{"type":"get_step","step":1860}
{"type":

{"type":"get_step","step":2087}
{"type":"get_step","step":2088}
{"type":"get_step","step":2089}
{"type":"get_step","step":2090}
{"type":"get_step","step":2091}
{"type":"get_step","step":2092}
{"type":"get_step","step":2093}
{"type":"get_step","step":2094}
{"type":"get_step","step":2095}
{"type":"get_step","step":2096}
{"type":"get_step","step":2097}
{"type":"get_step","step":2098}
{"type":"get_step","step":2099}
{"type":"get_step","step":2100}
{"type":"get_step","step":2101}
{"type":"get_step","step":2102}
{"type":"get_step","step":2103}
{"type":"get_step","step":2104}
{"type":"get_step","step":2105}
{"type":"get_step","step":2106}
{"type":"get_step","step":2107}
{"type":"get_step","step":2108}
{"type":"get_step","step":2109}
{"type":"get_step","step":2110}
{"type":"get_step","step":2111}
{"type":"get_step","step":2112}
{"type":"get_step","step":2113}
{"type":"get_step","step":2114}
{"type":"get_step","step":2115}
{"type":"get_step","step":2116}
{"type":"get_step","step":2117}
{"type":

{"type":"get_step","step":2344}
{"type":"get_step","step":2345}
{"type":"get_step","step":2346}
{"type":"get_step","step":2347}
{"type":"get_step","step":2348}
{"type":"get_step","step":2349}
{"type":"get_step","step":2350}
{"type":"get_step","step":2351}
{"type":"get_step","step":2352}
{"type":"get_step","step":2353}
{"type":"get_step","step":2354}
{"type":"get_step","step":2355}
{"type":"get_step","step":2356}
{"type":"get_step","step":2357}
{"type":"get_step","step":2358}
{"type":"get_step","step":2359}
{"type":"get_step","step":2360}
{"type":"get_step","step":2361}
{"type":"get_step","step":2362}
{"type":"get_step","step":2363}
{"type":"get_step","step":2364}
{"type":"get_step","step":2365}
{"type":"get_step","step":2366}
{"type":"get_step","step":2367}
{"type":"get_step","step":2368}
{"type":"get_step","step":2369}
{"type":"get_step","step":2370}
{"type":"get_step","step":2371}
{"type":"get_step","step":2372}
{"type":"get_step","step":2373}
{"type":"get_step","step":2374}
{"type":